In [2]:
import pandas as pd
import numpy as np

**Imputation of missing values**


In [3]:
train_data = pd.read_csv('/Users/nikak/Desktop/vubaby/DMT/data_DMT/training_set_VU_DM.csv')
test_data = pd.read_csv('/Users/nikak/Desktop/vubaby/DMT/data_DMT/test_set_VU_DM.csv')

In [4]:
# Missing values in the training data
missing_counts = train_data.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
print("\nMissing values in training data:")
#print(missing_counts)

missing_percent = (missing_counts / len(train_data)) * 100
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)
print("\nMissing values percentage in training data:")  
print(missing_percent)


Missing values in training data:

Missing values percentage in training data:
comp1_rate_percent_diff      98.095353
comp6_rate_percent_diff      98.060362
comp1_rate                   97.581250
comp1_inv                    97.387053
comp4_rate_percent_diff      97.356256
gross_bookings_usd           97.208949
comp7_rate_percent_diff      97.206428
comp6_rate                   95.156511
visitor_hist_starrating      94.920364
visitor_hist_adr_usd         94.897735
comp6_inv                    94.736633
comp4_rate                   93.800797
comp7_rate                   93.640058
srch_query_affinity_score    93.598552
comp4_inv                    93.069001
comp7_inv                    92.811677
comp3_rate_percent_diff      90.464625
comp2_rate_percent_diff      88.781786
comp8_rate_percent_diff      87.602118
comp5_rate_percent_diff      83.036706
comp3_rate                   69.056462
comp3_inv                    66.702814
comp8_rate                   61.344900
comp8_inv               

- 'visitor_hist_starrating' --> NULL signifies there is no purchase history of the customer
- 'visitor_hist_adr_usd' --> similar as above
- 'prop_review_score' --> info not available
- 'orig_destination_distance' --> info could not be calculated
- 'compN_rate' and 'compN_inv' --> no competitive data

In [5]:
import pandas as pd
import numpy as np

def impute_missing_values(df):
    # 1. Visitor history: -1 for no history, plus missing flag
    df['visitor_hist_starrating_missing'] = df['visitor_hist_starrating'].isnull().astype(int)
    df['visitor_hist_adr_usd_missing'] = df['visitor_hist_adr_usd'].isnull().astype(int)
    df['visitor_hist_starrating'] = df['visitor_hist_starrating'].fillna(-1)
    df['visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].fillna(-1)

    # 2. Property review score: -1 for unavailable + missing flag
    df['prop_review_score_missing'] = df['prop_review_score'].isnull().astype(int)
    df['prop_review_score'] = df['prop_review_score'].fillna(-1)

    # 3. Destination distance: impute with median + missing flag
    df['orig_destination_distance_missing'] = df['orig_destination_distance'].isnull().astype(int)
    df['orig_destination_distance'] = df['orig_destination_distance'].fillna(df['orig_destination_distance'].median())

    # 4. prop_location_score2: median impute + flag
    if 'prop_location_score2' in df.columns:
        df['prop_location_score2_missing'] = df['prop_location_score2'].isnull().astype(int)
        df['prop_location_score2'] = df['prop_location_score2'].fillna(df['prop_location_score2'].median())

    # 5. srch_query_affinity_score: fill with -1 + flag
    if 'srch_query_affinity_score' in df.columns:
        df['srch_query_affinity_score_missing'] = df['srch_query_affinity_score'].isnull().astype(int)
        df['srch_query_affinity_score'] = df['srch_query_affinity_score'].fillna(-1)

    return df
train = train_data.copy()
test = test_data.copy()

# ---------------------------
# Step 1: Handle visitor history fields
# ---------------------------
# visitor_cols = ['visitor_hist_starrating', 'visitor_hist_adr_usd']
# for col in visitor_cols:
#     train[f'{col}_missing'] = train[col].isnull().astype(int)
#     train[col].fillna(-1, inplace=True)  # -1 means no purchase history

# # ---------------------------
# # Step 2: Handle property review score
# # ---------------------------
# train['prop_review_score_missing'] = train['prop_review_score'].isnull().astype(int)
# train['prop_review_score'].fillna(-1, inplace=True)  # -1 means info not available

# # ---------------------------
# # Step 3: Handle orig_destination_distance
# # ---------------------------
# train['orig_destination_distance_missing'] = train['orig_destination_distance'].isnull().astype(int)
# train['orig_destination_distance'].fillna(train['orig_destination_distance'].median(), inplace=True)



# ---------------------------
#This is not needed anymore so I commented it out

# # Step 4: Handle compN_* features (rate, inv, and percent diff)
# # ---------------------------
# rate_cols = [f'comp{i}_rate' for i in range(1, 9)]
# inv_cols = [f'comp{i}_inv' for i in range(1, 9)]
# diff_cols = [f'comp{i}_rate_percent_diff' for i in range(1, 9)]

# # Fill rate and inv with 0 = "no competitive info" (neutral)
# train[rate_cols] = train[rate_cols].fillna(0)
# train[inv_cols] = train[inv_cols].fillna(0)

# # Leave diff as NaN so we can average it properly (ignoring missing)
# train[diff_cols] = train[diff_cols].apply(pd.to_numeric, errors='coerce')

# # ---------------------------
# # Step 5: Aggregate compN features into clean new features
# # ---------------------------
# train['expedia_cheaper_count'] = (train[rate_cols] == 1).sum(axis=1)
# train['comp_unavailable_count'] = (train[inv_cols] == 1).sum(axis=1)
# train['avg_comp_price_diff'] = train[diff_cols].mean(axis=1)

# # Optional: drop original compN_* columns
# train.drop(columns=rate_cols + inv_cols + diff_cols, inplace=True)

# ---------------------------
# Done!
# ---------------------------



**Applying Imputation to Train and Test data**

In [6]:
# Impute missing values
train = impute_missing_values(train)
test = impute_missing_values(test)

# Save to CSV
train.to_csv("train_imputed_CSV.csv", index=False)
test.to_csv("test_imputed_CSV.csv", index=False)

# Save to Pickle
train.to_pickle("train_imputed.pkl")
test.to_pickle("test_imputed.pkl")

print("Saved imputed train and test data.")


Saved imputed train and test data.


In [7]:
print("Missing value handling completed.")
print(train[['visitor_hist_starrating', 'visitor_hist_starrating_missing',
             'orig_destination_distance', 'orig_destination_distance_missing'
            #  , 'expedia_cheaper_count', 'avg_comp_price_diff'
            ]].head())

Missing value handling completed.
   visitor_hist_starrating  visitor_hist_starrating_missing  \
0                     -1.0                                1   
1                     -1.0                                1   
2                     -1.0                                1   
3                     -1.0                                1   
4                     -1.0                                1   

   orig_destination_distance  orig_destination_distance_missing  
0                      386.6                                  1  
1                      386.6                                  1  
2                      386.6                                  1  
3                      386.6                                  1  
4                      386.6                                  1  


In [ ]:
# Also not needed anymore
# # ---------------------------


# # Step 1: Define competitor column groups
# train = train_data.copy()
# rate_cols = [f'comp{i}_rate' for i in range(1, 9)]
# inv_cols = [f'comp{i}_inv' for i in range(1, 9)]
# diff_cols = [f'comp{i}_rate_percent_diff' for i in range(1, 9)]

# # Step 2: Fill missing values
# # Use np.nan for differences to allow proper mean calculation
# train[rate_cols] = train[rate_cols].fillna(0)       # 0 = same price (neutral)
# train[inv_cols] = train[inv_cols].fillna(0)         # 0 = available
# train[diff_cols] = train[diff_cols].fillna(np.nan)  # np.nan to allow correct average

# # Step 3: Create aggregated features
# train['cheaper_count'] = (train[rate_cols] == 1).sum(axis=1)
# train['comp_unavailable_count'] = (train[inv_cols] == 1).sum(axis=1)
# train['avg_comp_price_diff'] = train[diff_cols].mean(axis=1)

# # Step 4 (optional): Drop the original compX_* columns
# train.drop(columns=rate_cols + inv_cols + diff_cols, inplace=True)

# # Done!
# print("Competitor data processed and combined.")
# print(train[['cheaper_count', 'comp_unavailable_count', 'avg_comp_price_diff']].head())


Competitor data processed and combined.
   cheaper_count  comp_unavailable_count  avg_comp_price_diff
0              0                       0                  NaN
1              0                       1                  NaN
2              0                       0                  NaN
3              0                       1                  5.0
4              0                       0                  NaN


In [8]:
# Missing values in the training data
missing_counts = train.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
print("\nMissing values in training data:")
#print(missing_counts)

missing_percent = (missing_counts / len(train)) * 100
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)
print("\nMissing values percentage in training data:")  
print(missing_percent)


Missing values in training data:

Missing values percentage in training data:
comp1_rate_percent_diff    98.095353
comp6_rate_percent_diff    98.060362
comp1_rate                 97.581250
comp1_inv                  97.387053
comp4_rate_percent_diff    97.356256
gross_bookings_usd         97.208949
comp7_rate_percent_diff    97.206428
comp6_rate                 95.156511
comp6_inv                  94.736633
comp4_rate                 93.800797
comp7_rate                 93.640058
comp4_inv                  93.069001
comp7_inv                  92.811677
comp3_rate_percent_diff    90.464625
comp2_rate_percent_diff    88.781786
comp8_rate_percent_diff    87.602118
comp5_rate_percent_diff    83.036706
comp3_rate                 69.056462
comp3_inv                  66.702814
comp8_rate                 61.344900
comp8_inv                  59.916016
comp2_rate                 59.166392
comp2_inv                  57.036710
comp5_rate                 55.179155
comp5_inv                  52.403

In [2]:
import pandas as pd
test = pd.read_csv('test_imputed_CSV.csv')
# Missing values in the test data
missing_counts = test.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
print("\nMissing values in test data:")
#print(missing_counts)

missing_percent = (missing_counts / len(test)) * 100
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)
print("\nMissing values percentage in test data:")  
print(missing_percent)


Missing values in test data:

Missing values percentage in test data:
comp1_rate_percent_diff    98.175748
comp6_rate_percent_diff    98.041250
comp1_rate                 97.663405
comp1_inv                  97.481964
comp4_rate_percent_diff    97.315546
comp7_rate_percent_diff    97.190606
comp6_rate                 95.113510
comp6_inv                  94.693299
comp4_rate                 93.694102
comp7_rate                 93.633447
comp4_inv                  92.966422
comp7_inv                  92.806214
comp3_rate_percent_diff    90.498233
comp2_rate_percent_diff    88.836689
comp8_rate_percent_diff    87.679886
comp5_rate_percent_diff    83.063601
comp3_rate                 69.249269
comp3_inv                  66.905214
comp8_rate                 61.639064
comp8_inv                  60.217540
comp2_rate                 59.348929
comp2_inv                  57.225434
comp5_rate                 55.195826
comp5_inv                  52.395122
dtype: float64
